# Evaluating Different Optimizations of Networks

This notebook is used to compared different optimizations of networks
Load different networks and data and run evaluate function

In [ ]:
import time
import numpy as np
import tensorflow as tf

In [ ]:
@tf.function
def run(data, model):
    return model(data)

def evaluate(model, type, data):
    x_test, y_test = data
    prediction_digits = []
    if type.lower() == 'tflite':
        model.allocate_tensors()
        input_index = model.get_input_details()[0]["index"]
        output_index = model.get_output_details()[0]["index"]

        t1 = time.time()
        for test_image in x_test:
            # Pre-processing: add batch dimension and convert to float32 to match with
            # the model's input data format.
            test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
            model.set_tensor(input_index, test_image)

            # Run inference.
            model.invoke()

            # Post-processing: remove batch dimension and find the digit with highest
            # probability.
            output = model.get_tensor(output_index)
            # digit = np.argmax(output()[0])
            # prediction_digits.append(digit)
        t2 = time.time()-t1
    else:
        y_test = tf.keras.utils.to_categorical(y_test, 10)
        t1 = time.time()
        for test_image in x_test:
            test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
            # output = model(test_image)
            run(test_image, model)

        t2 = time.time()-t1
    
    print(f'{type} model time: {t2}')

In [ ]:
mnist = tf.keras.datasets.mnist
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


# original = tf.keras.models.load_model('functional_split_net')
# evaluate(original, 'normal', (x_test,y_test))
# del original

folded = tf.keras.models.load_model('Networks/functional_split_folded')
evaluate(folded, 'folded', (x_test, y_test))
del folded

tflite = interpreter = tf.lite.Interpreter(model_path='Networks/functional_base.tflite')
evaluate(tflite, 'tflite', (x_test, y_test))